In [105]:
import jieba
import jieba.posseg as pseg
import pandas as pd
import numpy as np
import tensorflow as tf
from collections import Counter
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
train_path = "../datasets/AutoMaster_TrainSet.csv"
test_path = "../datasets/AutoMaster_TestSet.csv"

In [7]:
df = pd.read_csv(train_path, encoding='utf-8')

In [8]:
df.head()

,QID,Brand,Model,Question,Dialogue,Report
0,Q1,奔驰,奔驰GL级,方向机重，助力泵，方向机都换了还是一样,技师说：[语音]|车主说：新的都换了|车主说：助力泵，方向机|技师说：[语音]|车主说：换了...,随时联系
1,Q2,奔驰,奔驰M级,奔驰ML500排气凸轮轴调节错误,技师说：你这个有没有电脑检测故障代码。|车主说：有|技师说：发一下|车主说：发动机之前亮故障...,随时联系
2,Q3,宝马,宝马X1(进口),2010款宝马X1，2011年出厂，2.0排量，通用6L45变速箱，原地换挡位PRND车辆闯...,技师说：你好，4缸自然吸气发动机N46是吧，先挂空档再挂其他档有没有闯动呢，变速箱油液位是否...,行驶没有顿挫的感觉，原地换挡有闯动，刹车踩重没有，这是力的限制的作用，应该没有问题
3,Q4,Jeep,牧马人,3.0V6发动机号在什么位置，有照片最好！,技师说：右侧排气管上方，缸体上靠近变速箱|车主说：[图片]|车主说：是不是这个？|车主说：这...,举起车辆，在左前轮这边的缸体上
4,Q5,奔驰,奔驰C级,2012款奔驰c180怎么样，维修保养，动力，值得拥有吗,技师说：家庭用车的话，还是可以入手的|技师说：维修保养费用不高|车主说：12年的180市场价...,家庭用车可以入手的，维修保养价格还可以。车况好，价格合理可以入手


### 分词并保存至csv文件

In [9]:
stop_words = '，：？。? ！! @ # $ % ^ & * ( ) [ ] { } > < = - + ~ ` --- (i (or / ; ;\' $1 |> \
                    --------- -------------------------------------------------------------------------- \
                    ========================= \
                    0 1 2 3 4 5 6 7 8 9 13 15 30 24 20 "a" tk> 95 45'

In [10]:
def process(s):
    seg = [i for i in jieba.cut(s) if i not in stop_words ]
    return " ".join(seg)

In [11]:
def build_vocab(df, sort=True, min_count=0, lower=False):
    data_columns = df.columns.tolist()
    data_seg = []
    df_new = pd.DataFrame()
    for col in data_columns:
        data_col = df[col]
        df[col] = df[col].apply(str)
        df_new[col] = df[col].apply(process)
    return df_new

In [12]:
df_new = build_vocab(df)
print(df_new.head())

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\86221\AppData\Local\Temp\jieba.cache
Loading model cost 0.638 seconds.
Prefix dict has been built succesfully.


  QID Brand     Model                                           Question  \
0  Q1    奔驰   奔驰 GL 级                         方向机 重 助力 泵 方向机 都 换 了 还是 一样   
1  Q2    奔驰    奔驰 M 级                              奔驰 ML500 排气 凸轮轴 调节 错误   
2  Q3    宝马  宝马 X1 进口  2010 款 宝马 X1 2011 年 出厂 2.0 排量 通用 6L45 变速箱 原地 换...   
3  Q4  Jeep       牧马人                       3.0 V6 发动机 号 在 什么 位置 有 照片 最好   
4  Q5    奔驰    奔驰 C 级                 2012 款 奔驰 c180 怎么样 维修保养 动力 值得 拥有 吗   

                                            Dialogue  \
0  技师 说 语音 车主 说 新 的 都 换 了 车主 说 助力 泵 方向机 技师 说 语音 车...   
1  技师 说 你 这个 有没有 电脑 检测 故障 代码 车主 说 有 技师 说 发 一下 车主 ...   
2  技师 说 你好 缸 自然 吸气 发动机 N46 是 吧 先挂 空档 再 挂 其他 档 有没有...   
3  技师 说 右侧 排气管 上方 缸体 上 靠近 变速箱 车主 说 图片 车主 说 是不是 这个...   
4  技师 说 家庭 用车 的话 还是 可以 入手 的 技师 说 维修保养 费用 不高 车主 说 ...   

                                              Report  
0                                              随时 联系  
1                                              随时 联系  
2  行驶 没有 顿挫 的 感觉 原地 换挡 有 闯动 刹车 踩 重 没有 这 是

In [13]:
df_new.to_csv('../datasets/train_set.seg.csv', index=False)

In [14]:
df_new.head()

,QID,Brand,Model,Question,Dialogue,Report
0,Q1,奔驰,奔驰 GL 级,方向机 重 助力 泵 方向机 都 换 了 还是 一样,技师 说 语音 车主 说 新 的 都 换 了 车主 说 助力 泵 方向机 技师 说 语音 车...,随时 联系
1,Q2,奔驰,奔驰 M 级,奔驰 ML500 排气 凸轮轴 调节 错误,技师 说 你 这个 有没有 电脑 检测 故障 代码 车主 说 有 技师 说 发 一下 车主 ...,随时 联系
2,Q3,宝马,宝马 X1 进口,2010 款 宝马 X1 2011 年 出厂 2.0 排量 通用 6L45 变速箱 原地 换...,技师 说 你好 缸 自然 吸气 发动机 N46 是 吧 先挂 空档 再 挂 其他 档 有没有...,行驶 没有 顿挫 的 感觉 原地 换挡 有 闯动 刹车 踩 重 没有 这 是 力 的 限制 ...
3,Q4,Jeep,牧马人,3.0 V6 发动机 号 在 什么 位置 有 照片 最好,技师 说 右侧 排气管 上方 缸体 上 靠近 变速箱 车主 说 图片 车主 说 是不是 这个...,举起 车辆 在 左 前轮 这边 的 缸体 上
4,Q5,奔驰,奔驰 C 级,2012 款 奔驰 c180 怎么样 维修保养 动力 值得 拥有 吗,技师 说 家庭 用车 的话 还是 可以 入手 的 技师 说 维修保养 费用 不高 车主 说 ...,家庭 用车 可以 入手 的 维修保养 价格 还 可以 车况 好 价格合理 可以 入手


In [15]:
df_read = pd.read_csv('../datasets/train_set.seg.csv')

In [16]:
df_read.head()

,QID,Brand,Model,Question,Dialogue,Report
0,Q1,奔驰,奔驰 GL 级,方向机 重 助力 泵 方向机 都 换 了 还是 一样,技师 说 语音 车主 说 新 的 都 换 了 车主 说 助力 泵 方向机 技师 说 语音 车...,随时 联系
1,Q2,奔驰,奔驰 M 级,奔驰 ML500 排气 凸轮轴 调节 错误,技师 说 你 这个 有没有 电脑 检测 故障 代码 车主 说 有 技师 说 发 一下 车主 ...,随时 联系
2,Q3,宝马,宝马 X1 进口,2010 款 宝马 X1 2011 年 出厂 2.0 排量 通用 6L45 变速箱 原地 换...,技师 说 你好 缸 自然 吸气 发动机 N46 是 吧 先挂 空档 再 挂 其他 档 有没有...,行驶 没有 顿挫 的 感觉 原地 换挡 有 闯动 刹车 踩 重 没有 这 是 力 的 限制 ...
3,Q4,Jeep,牧马人,3.0 V6 发动机 号 在 什么 位置 有 照片 最好,技师 说 右侧 排气管 上方 缸体 上 靠近 变速箱 车主 说 图片 车主 说 是不是 这个...,举起 车辆 在 左 前轮 这边 的 缸体 上
4,Q5,奔驰,奔驰 C 级,2012 款 奔驰 c180 怎么样 维修保养 动力 值得 拥有 吗,技师 说 家庭 用车 的话 还是 可以 入手 的 技师 说 维修保养 费用 不高 车主 说 ...,家庭 用车 可以 入手 的 维修保养 价格 还 可以 车况 好 价格合理 可以 入手


### 使用Word2Vec训练词向量

In [155]:
def build_dataset(path):
    print('Read data, path:{0}'.format(path))
    train = pd.read_csv(path, encoding='utf-8')
    lines = []
    for k in ['Brand', 'Model', 'Question', 'Dialogue', 'Report']:
        a = list(train[k].apply(str).values)
        print(a)
        
        b = [i.split(' ') for i in a]
        lines.extend(b)

    return lines

In [156]:
train_seg_path = "train_set.seg.csv"
train_texts = build_dataset(train_seg_path)
print(train_texts)

Read data, path:train_set.seg.csv
['奔驰', '奔驰', '宝马', 'Jeep', '奔驰', '雪佛兰', '丰田', '昌河', '长安', '吉利 汽车', '五菱 汽车', '丰田', '福特', '本田', '奇瑞', '日产', '铃木', '现代', '宝马', '长城', '现代', '雪佛兰', '荣威', '现代', '现代', '丰田', '长安', '北汽 幻速', '别克', '奥迪', '江淮', '马自达', '长安 欧尚', '现代', '起亚', '长安', '荣威', '大众', '北京', '众泰', '吉利 汽车', '奥迪', '捷豹', '哈弗', '丰田', '起亚', '奥迪', '大众', '别克', '本田', '丰田', 'nan', '标致', '雪铁龙', 'nan', '雪铁龙', '猎豹 汽车', '标致', '雪铁龙', '雪铁龙', '铃木', '吉利 汽车', '标致', '本田', '沃尔沃', '奔驰', '大众', '大众', '福特', '斯柯达', '别克', '大众', '大众', '福特', '别克', '奥迪', '福特', '丰田', '起亚', '别克', '马自达', '大众', '五菱 汽车', '保时捷', '长安', '东风 风光', '宝马', '宝马', '荣威', '斯柯达', '五菱 汽车', '现代', '双龙', '纳智捷', '大众', '大众', '众泰', '别克', '一汽', '雪佛兰', '别克', '宝骏', '大众', '奥迪', '瑞麒', '别克', '大众', '吉利 汽车', '别克', '东风', '比亚迪', '斯柯达', '大众', '雪佛兰', '广汽传祺', '奔驰', '雪佛兰', '别克', '别克', '东风 风神', '福特', '雪佛兰', '大众', '上汽 大通', '五菱 汽车', '别克', '起亚', '雪佛兰', '长安 欧尚', '华泰', '别克', '大众', '中华', '哈弗', '长安', '雪铁龙', '别克', '阿尔法 · 罗密欧', '福特', '启辰', '奔驰', '本田', '吉利 汽车', '雪佛兰', '福田 乘用车', '雪铁龙', '

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
w2v_bin_path = "../w2v.bin"

In [83]:
embedding_size = 100

In [158]:
def build(train_vocab, out_path=None, embedding_size=100, sentence_path='',
          w2v_bin_path="w2v.bin", min_count=5, col_sep='\t'):
#     sentences = extract_sentence(train_seg_path, test_seg_path, col_sep=col_sep)
#     save_sentence(sentences, sentence_path)
    print('train w2v model...')
    # train model
    w2v = Word2Vec(sg=1, sentences=train_vocab,
                   size=embedding_size, window=5, min_count=min_count, iter=40)
    w2v.wv.save_word2vec_format(w2v_bin_path, binary=True)
    print("save %s ok." % w2v_bin_path)
    # test
    sim = w2v.wv.similarity('大师几笔', '技师')
    print('大 vs 小 similarity score:', sim)


In [159]:
build(train_texts, w2v_bin_path=w2v_bin_path, min_count=3)

train w2v model...


D:\Anaconda3\envs\tensorflow2.0\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


save w2v.bin ok.
大 vs 小 similarity score: 0.252085


In [18]:
model = KeyedVectors.load_word2vec_format(w2v_bin_path, binary=True)

In [19]:
model.most_similar("方向机")

[('向机', 0.6581052541732788),
 ('管柱', 0.624971866607666),
 ('方向', 0.6190668940544128),
 ('管住', 0.6180574893951416),
 ('转向', 0.6063190698623657),
 ('助力', 0.5997176170349121),
 ('拉杆', 0.5992100238800049),
 ('十字街', 0.5878870487213135),
 ('个球头', 0.5866455435752869),
 ('拉球头', 0.5850415825843811)]

In [20]:
model.index2word[3]

'车主'

In [104]:
model.word_vec('车主')

array([ 0.04905931, -0.15363947, -0.13510936,  0.07078981, -0.19028786,
        0.18019754, -0.03220895, -0.03691961,  0.04184718, -0.0809982 ,
       -0.08869021,  0.06783617, -0.08706257, -0.02917323, -0.01936564,
        0.03449244, -0.08805738,  0.17681025,  0.04012203,  0.07957872,
       -0.14918026,  0.01711098, -0.05900019, -0.11269657,  0.05427176,
       -0.05779349, -0.03453308, -0.00902537,  0.18193395,  0.15395658,
       -0.09681128,  0.04692857, -0.03964363,  0.1280545 ,  0.08088426,
        0.02698058, -0.0707665 , -0.10947863, -0.17911583, -0.04730757,
       -0.06489594, -0.00201858, -0.08343294,  0.0207491 ,  0.00704715,
       -0.11721613,  0.18242875, -0.09062335, -0.05901468,  0.10132237,
       -0.09789021, -0.1516594 ,  0.0276893 , -0.061536  ,  0.12511925,
        0.07608558,  0.02721855, -0.00311152,  0.05981553,  0.00558488,
       -0.05963785,  0.01927007,  0.02087756,  0.09828558, -0.0067789 ,
       -0.03100376, -0.1689943 , -0.01791076, -0.11596143,  0.03

In [22]:
model.vectors[3]

array([ 0.04905931, -0.15363947, -0.13510936,  0.07078981, -0.19028786,
        0.18019754, -0.03220895, -0.03691961,  0.04184718, -0.0809982 ,
       -0.08869021,  0.06783617, -0.08706257, -0.02917323, -0.01936564,
        0.03449244, -0.08805738,  0.17681025,  0.04012203,  0.07957872,
       -0.14918026,  0.01711098, -0.05900019, -0.11269657,  0.05427176,
       -0.05779349, -0.03453308, -0.00902537,  0.18193395,  0.15395658,
       -0.09681128,  0.04692857, -0.03964363,  0.1280545 ,  0.08088426,
        0.02698058, -0.0707665 , -0.10947863, -0.17911583, -0.04730757,
       -0.06489594, -0.00201858, -0.08343294,  0.0207491 ,  0.00704715,
       -0.11721613,  0.18242875, -0.09062335, -0.05901468,  0.10132237,
       -0.09789021, -0.1516594 ,  0.0276893 , -0.061536  ,  0.12511925,
        0.07608558,  0.02721855, -0.00311152,  0.05981553,  0.00558488,
       -0.05963785,  0.01927007,  0.02087756,  0.09828558, -0.0067789 ,
       -0.03100376, -0.1689943 , -0.01791076, -0.11596143,  0.03

In [23]:
model.index2entity[3]

'车主'

### Convert Sentences to Ids

In [24]:
def word2index(model):
    """
    建立词-->索引字典
    """
    word2index = {}
    for i in range(len(model.vocab)):
        word = model.index2word[i]
        word2index[word] = i
    return word2index

In [25]:
word2index_dic = word2index(model)

In [26]:
len(word2index_dic)

107579

In [27]:
word2index_dic['<start>'] = len(word2index_dic)
word2index_dic['<end>'] = len(word2index_dic) + 1

In [28]:
len(word2index_dic)

107581

In [29]:
def process_split(sen):
    """将句子转为ids
    """
    sen_list = []
    if sen is not None:
        sen_list = [str(word2index_dic[i]) for i in str(sen).split(' ')]
    return ' '.join(sen_list)

In [30]:
process_split(df_read['Question'][1])

'376 29601 492 298 465 1854'

In [31]:
df_read['Question'][1]

'奔驰 ML500 排气 凸轮轴 调节 错误'

In [32]:
len(df_read['Question'])

82943

In [33]:
df_read['Question'].apply(process_split)

0                      505 1062 324 315 505 21 28 4 42 117
1                               376 29601 492 298 465 1854
2        1695 151 216 2926 1494 126 1343 648 787 1514 5...
3                     2007 4037 26 403 16 27 106 6 650 238
4            1549 151 376 12465 520 4304 289 2378 15751 12
                               ...                        
82938                          732 322 418 1 268 418 74 53
82939    1467 151 802 5839 6444 74 995 1 480 352 39 273...
82940    292 1057 100 239 594 394 547 6 2411 780 1741 2...
82941    12260 3466 2243 17907 19868 71845 26 9042 52 1...
82942    20 8 1 5 3973 126 760 807 554 2037 963 1 413 3...
Name: Question, Length: 82943, dtype: object

In [37]:
df.columns

Index(['QID', 'Brand', 'Model', 'Question', 'Dialogue', 'Report'], dtype='object')

In [53]:
def convertSentences2Ids(df, model):
    for col in ['Brand', 'Model', 'Question', 'Dialogue', 'Report']:
        df_col = df[col].apply(process_split)
        df[col + '_' + 'index'] = df_col
    return df

In [54]:
df_index = convertSentences2Ids(df_read, model)

### 实现Seq2Seq

In [55]:
df_index.head()

,QID,Brand,Model,Question,Dialogue,Report,Brand_index,Model_index,Question_index,Dialogue_index,Report_index
0,Q1,奔驰,奔驰 GL 级,方向机 重 助力 泵 方向机 都 换 了 还是 一样,技师 说 语音 车主 说 新 的 都 换 了 车主 说 助力 泵 方向机 技师 说 语音 车...,随时 联系,376,376 3238 932,505 1062 324 315 505 21 28 4 42 117,2 0 18 3 0 156 1 21 28 4 3 0 324 315 505 2 0 1...,622 545
1,Q2,奔驰,奔驰 M 级,奔驰 ML500 排气 凸轮轴 调节 错误,技师 说 你 这个 有没有 电脑 检测 故障 代码 车主 说 有 技师 说 发 一下 车主 ...,随时 联系,376,376 3421 932,376 29601 492 298 465 1854,2 0 14 7 70 52 152 22 477 3 0 6 2 0 619 24 3 0...,622 545
2,Q3,宝马,宝马 X1 进口,2010 款 宝马 X1 2011 年 出厂 2.0 排量 通用 6L45 变速箱 原地 换...,技师 说 你好 缸 自然 吸气 发动机 N46 是 吧 先挂 空档 再 挂 其他 档 有没有...,行驶 没有 顿挫 的 感觉 原地 换挡 有 闯动 刹车 踩 重 没有 这 是 力 的 限制 ...,216,216 2926 777,1695 151 216 2926 1494 126 1343 648 787 1514 5...,2 0 20 180 1408 1680 26 10125 5 47 8528 1580 1...,120 15 589 1 119 892 513 6 7464 139 218 1062 1...
3,Q4,Jeep,牧马人,3.0 V6 发动机 号 在 什么 位置 有 照片 最好,技师 说 右侧 排气管 上方 缸体 上 靠近 变速箱 车主 说 图片 车主 说 是不是 这个...,举起 车辆 在 左 前轮 这边 的 缸体 上,2774,5449,2007 4037 26 403 16 27 106 6 650 238,2 0 1105 428 2586 1293 76 2306 62 3 0 33 3 0 8...,5357 95 16 499 510 748 1 1293 76
4,Q5,奔驰,奔驰 C 级,2012 款 奔驰 c180 怎么样 维修保养 动力 值得 拥有 吗,技师 说 家庭 用车 的话 还是 可以 入手 的 技师 说 维修保养 费用 不高 车主 说 ...,家庭 用车 可以 入手 的 维修保养 价格 还 可以 车况 好 价格合理 可以 入手,376,376 1882 932,1549 151 376 12465 520 4304 289 2378 15751 12,2 0 5113 583 56 42 13 1874 1 2 0 4304 402 1793...,5113 583 13 1874 1 4304 253 93 13 1927 17 9106...


In [56]:
df_index.shape

(82943, 11)

In [57]:
df_index.dropna(axis=0, how='any', inplace=True)

In [58]:
df_index.shape

(81559, 11)

In [59]:
#合并除report的字段
df_index['input'] = df_index['Brand'] + ' ' + df_index['Model'] + ' ' + df_index['Question'] + ' ' + df_index['Dialogue']
df_index['input_index'] = df_index['Brand_index'] + ' ' + df_index['Model_index'] + ' ' + df_index['Question_index'] + ' ' + df_index['Dialogue_index']
df_index.drop(['Brand', 'Model', 'Question', 'Dialogue'], axis=1, inplace=True)
df_index.drop(['Brand_index', 'Model_index', 'Question_index', 'Dialogue_index'], axis=1, inplace=True)

In [60]:
df_index.head()

,QID,Report,Report_index,input,input_index
0,Q1,随时 联系,622 545,奔驰 奔驰 GL 级 方向机 重 助力 泵 方向机 都 换 了 还是 一样 技师 说 语音 ...,376 376 3238 932 505 1062 324 315 505 21 28 4 ...
1,Q2,随时 联系,622 545,奔驰 奔驰 M 级 奔驰 ML500 排气 凸轮轴 调节 错误 技师 说 你 这个 有没有 ...,376 376 3421 932 376 29601 492 298 465 1854 2 ...
2,Q3,行驶 没有 顿挫 的 感觉 原地 换挡 有 闯动 刹车 踩 重 没有 这 是 力 的 限制 ...,120 15 589 1 119 892 513 6 7464 139 218 1062 1...,宝马 宝马 X1 进口 2010 款 宝马 X1 2011 年 出厂 2.0 排量 通用 6...,216 216 2926 777 1695 151 216 2926 1494 126 13...
3,Q4,举起 车辆 在 左 前轮 这边 的 缸体 上,5357 95 16 499 510 748 1 1293 76,Jeep 牧马人 3.0 V6 发动机 号 在 什么 位置 有 照片 最好 技师 说 右侧 ...,2774 5449 2007 4037 26 403 16 27 106 6 650 238...
4,Q5,家庭 用车 可以 入手 的 维修保养 价格 还 可以 车况 好 价格合理 可以 入手,5113 583 13 1874 1 4304 253 93 13 1927 17 9106...,奔驰 奔驰 C 级 2012 款 奔驰 c180 怎么样 维修保养 动力 值得 拥有 吗 技...,376 376 1882 932 1549 151 376 12465 520 4304 2...


In [72]:
def preprocess_word(w):
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

def preprocess_id(w):
    w = str(len(word2index_dic)) + ' ' + w + ' ' + str(len(word2index_dic) + 1)
    return w

In [74]:
import re

In [75]:
#添加<start><end>
df_index['input'] = df_index['input'].apply(preprocess_word).copy()
df_index['input_index'] = df_index['input_index'].apply(preprocess_id).copy()

In [77]:
df_index.head()

,QID,Report,Report_index,input,input_index
0,Q1,随时 联系,622 545,<start> 奔驰 奔驰 GL 级 方向机 重 助力 泵 方向机 都 换 了 还是 一样 ...,107581 376 376 3238 932 505 1062 324 315 505 2...
1,Q2,随时 联系,622 545,<start> 奔驰 奔驰 M 级 奔驰 ML500 排气 凸轮轴 调节 错误 技师 说 你...,107581 376 376 3421 932 376 29601 492 298 465 ...
2,Q3,行驶 没有 顿挫 的 感觉 原地 换挡 有 闯动 刹车 踩 重 没有 这 是 力 的 限制 ...,120 15 589 1 119 892 513 6 7464 139 218 1062 1...,<start> 宝马 宝马 X1 进口 2010 款 宝马 X1 2011 年 出厂 2 ....,107581 216 216 2926 777 1695 151 216 2926 1494...
3,Q4,举起 车辆 在 左 前轮 这边 的 缸体 上,5357 95 16 499 510 748 1 1293 76,<start> Jeep 牧马人 3 . 0 V6 发动机 号 在 什么 位置 有 照片 最...,107581 2774 5449 2007 4037 26 403 16 27 106 6 ...
4,Q5,家庭 用车 可以 入手 的 维修保养 价格 还 可以 车况 好 价格合理 可以 入手,5113 583 13 1874 1 4304 253 93 13 1927 17 9106...,<start> 奔驰 奔驰 C 级 2012 款 奔驰 c180 怎么样 维修保养 动力 值...,107581 376 376 1882 932 1549 151 376 12465 520...


In [110]:
max_words_size = 30000
max_input_size = 500
max_target_size = 50

def tokenize(texts):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', num_words=max_words_size)
    tokenizer.fit_on_texts(texts)
    
    word_index = tokenizer.word_index
    tensor = tf.keras.preprocessing.sequence.pad_sequences(texts, max_input_size, padding='post')
    return tensor, word_index, tokenizer

In [125]:
input_data = [str(i) for i in df_index['input'].values.tolist()]
input_data

['<start> 奔驰 奔驰 GL 级 方向机 重 助力 泵 方向机 都 换 了 还是 一样 技师 说 语音 车主 说 新 的 都 换 了 车主 说 助力 泵 方向机 技师 说 语音 车主 说 换 了 方向机 带 的 有 车主 说 图片 技师 说 语音 车主 说 有 助力 就是 重 这车 要 匹配 吧 技师 说 不 需要 技师 说 你 这 是 更换 的 部件 有 问题 车主 说 跑 快 了 还好 点 就 倒车 重 的 很 技师 说 是 非常 重 吗 车主 说 是 的 累人 技师 说 语音 车主 说 我 觉得 也 是 可是 车主 是 以前 没 这么 重 选 吧 助理 泵 换 了 不行 又 把 放 向 机换 了 现在 还 这样 就 不 知道 咋 和 车主 解释 技师 说 语音 技师 说 语音 <end>',
 '<start> 奔驰 奔驰 M 级 奔驰 ML500 排气 凸轮轴 调节 错误 技师 说 你 这个 有没有 电脑 检测 故障 代码 车主 说 有 技师 说 发 一下 车主 说 发动机 之前 亮 故障 灯 、 显示 是 失火 、 有点 缺缸 、 现在 又 没有 故障 、 发动机 多少 有点 抖动 、 检查 先前 的 故障 是 报 这个 故障 车主 说 稍 等 车主 说 显示 图片 太大传 不了 技师 说 语音 车主 说 这个 对 发动机 的 抖动 、 失火 、 缺缸 有 直接 联系 吗 技师 说 语音 车主 说 还有 就是 报 （ 左右 排气 凸轮轴 作动 电磁铁 ） 对 正极 短路 、 对 地 短路 、 对 导线 断路 技师 说 语音 车主 说 这 几个 电磁阀 和 问 您 的 第一个 故障 有 直接 关系 吧 技师 说 语音 车主 说 这个 有 办法 检测 它 好坏 吗 技师 说 语音 车主 说 谢谢 技师 说 不 客气 <end>',
 '<start> 宝马 宝马 X1 进口 2010 款 宝马 X1 2011 年 出厂 2 . 0 排量 通用 6L45 变速箱 原地 换挡 位 PRND 车辆 闯动 行驶 升降 档 正常 轮离 地 换挡 无 冲击 感 更换 变速箱 油 12L 无 改变 试 过 一辆 2014 年 进口 X1 原地 换挡 位 也 有 冲击 感 这是 什么 情况 哪里 的 问题 技师 说 你好 缸 自然 吸气 发动机 N46 是 吧 先挂 

In [122]:
def get_embedding(df, word_num, path='', embed_size=100):
    input_data = [str(i) for i in df['input'].values.tolist()]
    target_data = [str(i) for i in df['Report'].values.tolist()]

    tensor_input, word_index_input, tokenizer_input = tokenize(input_data)
    tensor_target, word_index_target, tokenizer_target = tokenize(target_data)

    encoder_embedding = np.zeros((word_num, embed_size))
    for word, id in word_index_input.items():
        if word not in model.vocab:
            word_vec = np.random.uniform(0.25, 0.25, embed_size)
        else:
            word_vec = model.word_vec(word)
        encoder_embedding[id] = word_vec
    return encoder_embedding

In [123]:
get_embedding(df_index, len(model.vocab), embed_size=100)

ValueError: invalid literal for int() with base 10: '<start> 奔驰 奔驰 GL 级 方向机 重 助力 泵 方向机 都 换 了 还是 一样 技师 说 语音 车主 说 新 的 都 换 了 车主 说 助力 泵 方向机 技师 说 语音 车主 说 换 了 方向机 带 的 有 车主 说 图片 技师 说 语音 车主 说 有 助力 就是 重 这车 要 匹配 吧 技师 说 不 需要 技师 说 你 这 是 更换 的 部件 有 问题 车主 说 跑 快 了 还好 

In [234]:
data_input[170000]

['216', '17104', '229', '74', '140']

In [235]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(data_input, data_target, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

ValueError: Found input variables with inconsistent numbers of samples: [331772, 82943]

In [ ]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train) // BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index) + 1
vocab_tar_size = len(targ_lang.word_index) + 1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)